In [1]:
import pandas as pd
import geopandas as gpd
import re
import os

In [2]:
# Descomentar y cambiar directorio si se usa aparte. Cambiar directorio
#os.chdir(r'C:\Users\matia\Documents\Posts\Dengue BA')

In [3]:
with open('Dengue Casos BA.txt', encoding = 'utf-8') as file:
    partidos = file.readlines() #Lee las líneas

In [4]:
#Defino las regex
reNombre = re.compile(r'\D+')
reUltNum = re.compile(r'\d+\n')

In [5]:
#Creo el diccionario con los datos
casos = {}
for partido in partidos:
    moNom = reNombre.search(partido)
    moUltNum = reUltNum.search(partido)
    try:
        casos[moNom.group().strip()] = int(moUltNum.group()[:-1]) #Strip para sacar el espacio al final. int([:-1]) para sacar el \n y convertir a int
        
    except:
        pass

In [6]:
#IMPORTANTE: Esta parte funciona solamente para Python 3.7+, ya que depende de la conservación del orden del diccionario
dicDF = {'departamento': [i for i in casos.keys()], 'Casos_dengue':[i for i in casos.values()]} #Lo pytónico es porque es un objeto, no exactamente una lista.
df = pd.DataFrame(dicDF)

In [11]:
#Carga el geopackage y le agrega los casos de dengue de la tabla.
provNBI = gpd.read_file('NBI.gpkg')
provNBI = provNBI.merge(df, on='departamento', how = 'outer').fillna(0)
provNBI

,link,departamento,personas_con_nbi,personas_con_nbi_porc,total_pob,hogares_con_nbi,hogares_con_nbi_porc,total_hog,geometry,Casos_dengue
0,06462,Leandro N. Alem,1107,6.6,16662,259,4.5,5783,(POLYGON ((-61.64929720103974 -34.321900122777...,0.0
1,06147,Carlos Casares,1557,7.1,21907,328,4.2,7775,(POLYGON ((-61.53275878648711 -35.380107727236...,0.0
2,06028,Almirante Brown,74914,13.6,550349,16497,10.5,156918,(POLYGON ((-58.31263586612462 -34.875870138115...,17.0
3,06035,Avellaneda,26759,7.9,340276,6609,5.8,113142,(POLYGON ((-58.33444256953027 -34.630177109406...,37.0
4,06091,Berazategui,43990,13.7,321616,9742,10.5,93164,(POLYGON ((-58.01701653995696 -34.781198123038...,18.0
5,06260,Esteban Echeverría,41007,13.7,300273,9279,10.8,85952,(POLYGON ((-58.50223056019455 -34.722856162321...,29.0
6,06270,Ezeiza,28787,18.0,160261,6274,14.1,44487,(POLYGON ((-58.52072608709163 -34.742176439217...,7.0
7,06274,Florencio Varela,87790,20.8,421822,19295,17.1,113135,(POLYGON ((-58.31263586612462 -34.875870138115...,12.0
8,06371,General San Martín,37861,9.2,411814,8961,6.7,133202,(POLYGON ((-58.60052214022971 -34.491188470590...,31.0
9,06408,Hurlingham,16209,9.0,180367,3799,6.9,55122,(POLYGON ((-58.69391065085578 -34.594713298568...,6.0


In [16]:
provNBI['porc_personas_NBI'] = 100*provNBI['personas_con_nbi'] / provNBI['total_pob']
provNBI['porc_hogares_NBI'] = 100*provNBI['hogares_con_nbi'] / provNBI['total_hog']
provNBI['Casos_100.000_Hab'] = 100000 * provNBI['Casos_dengue'] / provNBI['total_hog']

In [1]:
provNBI.to_file("NBI_Dengue_BsAs.gpkg", layer='dengue', driver="GPKG")